In [2]:
%tensorflow_version 1.x
#!git clone https://github.com/tensorflow/cleverhans.git
!pip install cleverhans
!pip install adversarial-robustness-toolbox

TensorFlow 1.x selected.
     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 163kB 59.3MB/s 
     |████████████████████████████████| 491kB 2.8MB/s 
     |████████████████████████████████| 7.1MB 13.1MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import keras
import numpy as np
import matplotlib.pyplot as plt
from art.classifiers import KerasClassifier
from art.utils import load_dataset
import random
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.disable_eager_execution()

from cleverhans.compat import softmax_cross_entropy_with_logits
from cleverhans.utils_keras import KerasModelWrapper



# Load the CIFAR-10 Dataset
(x_train_victim, y_train_victim), (x_test_victim, y_test_victim), min_, max_ = load_dataset(str("cifar10")) # Original Dataset
print("x_train_victim shape: " + str(x_train_victim.shape) + "\n" + "x_train_victim size: " + str(x_train_victim.size) + "\n" + # this print statement is used for understanding what the CIFAR-10 dataset is
      "y_train_victim shape: " + str(y_train_victim.shape) + "\n" + "y_train_victim size: " + str(y_train_victim.size) + "\n" +
      "x_test_victim shape: " + str(x_test_victim.shape) + "\n" + "x_test_victim size: " + str(x_test_victim.size) + "\n" +
      "y_test_victim shape: " + str(y_test_victim.shape) + "\n" + "y_test_victim size: " + str(y_test_victim.size) + "\n")
print()


#  Load the victim model
classifier_url ="https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1" #@param {type:"string"} # model is downloaded from this site
IMAGE_SHAPE = (32, 32) # the image shape is needed so that the model knows the input-shape and since we are working with the CIFAR-10 all the images are 32 x 32 color images
victim_classifier = KerasClassifier(model=tf.keras.Sequential([hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))]), clip_values=(min_, max_)) # this bascially creates a keras wrapper around the downloaded model so that we can use it with keras functions.


# Evaluating the victim model on the benign dataset
predictions = victim_classifier.predict(x_test_victim) # giving the classifier the x_test of the CIFAR-10 dataset.
accuracy_benign = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_victim, axis=1)) / len(y_test_victim) # calculates the accuracy of the predictions
print("Accuracy on benign test examples for victim classifier: {}%\n".format(accuracy_benign * 100))


Using TensorFlow backend.



x_train_victim shape: (50000, 32, 32, 3)
x_train_victim size: 153600000
y_train_victim shape: (50000, 10)
y_train_victim size: 500000
x_test_victim shape: (10000, 32, 32, 3)
x_test_victim size: 30720000
y_test_victim shape: (10000, 10)
y_test_victim size: 100000


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Accuracy on benign test examples for victim classifier: 94.52000000000001%



In [4]:
#  Collect subset
def exract_subset(data, labels):
    x_pre = []  
    y_pre = []  
    #count = 0
    for index in range(0, len(data)):  
        x_pre.append(data[index])  
        y_predict = np.argmax(victim_classifier.predict( data[index].reshape( (1, data[index].shape[ 0 ], data[index].shape[ 1 ], data[index].shape[ 2 ]) ) ) )  # add the image label to the y_test set
        y_pre.append(y_predict)
       
    x = np.asarray(x_pre)  
    y = keras.utils.to_categorical(np.asarray(y_pre), 10)  
    return x, y

X_subset, Y_subset = exract_subset( x_test_victim, y_test_victim )
print("X_subset shape: " + str(X_subset.shape) + "\n" + "X_subset size: " + str(X_subset.size) + "\n" +
      "Y_subset shape: " + str(Y_subset.shape) + "\n" + "Y_subset size: " + str(Y_subset.size) + "\n")

np.save('/content/gdrive/My Drive/X_subset', X_subset) 
np.save('/content/gdrive/My Drive/Y_subset', Y_subset) 

X_subset shape: (10000, 32, 32, 3)
X_subset size: 30720000
Y_subset shape: (10000, 10)
Y_subset size: 100000



Up to this point the victim classifer has been evaluted on the test set of cifar10 = 94.5% accuracy, and the test set has been extracted with the labels given by the victim model's predtiction, i.e. 10000 examples

In [5]:
#  Loading the subset data
X = np.load('/content/gdrive/My Drive/X_subset.npy') 
Y = np.load('/content/gdrive/My Drive/Y_subset.npy') 
print("X shape: " + str(X.shape) + "\n" + "X size: " + str(X.size) + "\n" + 
      "Y shape: " + str(Y.shape) + "\n" + "Y size: " + str(Y.size) + "\n")
print()

x_train_substitute = X[:7500]
y_train_substitute = Y[:7500]
print("x_train_substitute shape: " + str(x_train_substitute.shape) + "\n" + "x_train_substitute size: " + str(x_train_substitute.size) + "\n" + 
      "y_train_substitute shape: " + str(y_train_substitute.shape) + "\n" + "y_train_substitute size: " + str(y_train_substitute.size) + "\n")
print()

x_test_substitute = X[7500:]
y_test_substitute = Y[7500:]
print("x_test_substitute shape: " + str(x_test_substitute.shape) + "\n" + "x_test_substitute size: " + str(x_test_substitute.size) + "\n" + 
      "y_test_substitute shape: " + str(y_test_substitute.shape) + "\n" + "y_test_substitute size: " + str(y_test_substitute.size) + "\n")
print()


#  Creating the model
model_substitute = Sequential()
model_substitute.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train_substitute.shape[1:]))
model_substitute.add(Activation("relu"))
model_substitute.add(Conv2D(32, (3, 3)))
model_substitute.add(Activation("relu"))
model_substitute.add(MaxPooling2D(pool_size=(2, 2)))
model_substitute.add(Dropout(0.25))

model_substitute.add(Conv2D(64, (3, 3), padding="same"))
model_substitute.add(Activation("relu"))
model_substitute.add(Conv2D(64, (3, 3)))
model_substitute.add(Activation("relu"))
model_substitute.add(MaxPooling2D(pool_size=(2, 2)))
model_substitute.add(Dropout(0.25))

model_substitute.add(Flatten())
model_substitute.add(Dense(512))
model_substitute.add(Activation("relu"))
model_substitute.add(Dropout(0.5))
model_substitute.add(Dense(10))
model_substitute.add(Activation("softmax"))

model_substitute.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])



# Step 3: Creating the classifier
substitute_classifier = KerasClassifier(model=model_substitute, clip_values=(0., 1.))


X shape: (10000, 32, 32, 3)
X size: 30720000
Y shape: (10000, 10)
Y size: 100000


x_train_substitute shape: (7500, 32, 32, 3)
x_train_substitute size: 23040000
y_train_substitute shape: (7500, 10)
y_train_substitute size: 75000


x_test_substitute shape: (2500, 32, 32, 3)
x_test_substitute size: 7680000
y_test_substitute shape: (2500, 10)
y_test_substitute size: 25000





In [6]:
def one_hot(value):
  vec = np.zeros((10))
  vec[value] = 1
  return vec

def jacobian(predictions, inputs, num_classes):
    #That is, how does the kth element of yhat vary wrt x?
    return [tf.gradients(predictions[:, c], inputs)[0] for c in range(0, num_classes)]

def jacobian_prediction_dimension(grads, predictions):
    return [grads[predictions[i]][i] for i in np.arange(len(predictions))]

wrapper = KerasModelWrapper( model_substitute ) # create keras wrapper for substitute model
adv_train_epochs = 1 # run the loop for 2 runs genreating 22500 samples from 7500 samples
adv_train_set = x_train_substitute / 255.0 # assign the varaible adv_train_set the x_train which holds 7500 samples


for adv_train_epoch in range(adv_train_epochs): # loop for syntehtic data generation
    print("RUN: " + str(adv_train_epoch))
    print("Before: ")
    print(adv_train_set.shape) # initial size of the x_train (7500, 32, 32, 3)

    # Get labels from victim model and train substitute model
    oracle_labels = victim_classifier.predict(adv_train_set) # have the victim model label the x_train values, i.e. getting the y_train labels = 7500 labels
    print("Oracle Predict: ")
    print(oracle_labels.shape) # shape of the y_train = (7500, 10)
    substitute_classifier.fit(adv_train_set, oracle_labels, nb_epochs=10, batch_size=128) # fit the substitute model with the training set created thus far.    

    # Convert the labels from victim model to one hot encoded vectors
    oracle_labels = np.zeros((adv_train_set.shape[0],10))
    for i in range(0,x_train_substitute.shape[0]):
        oracle_labels[i] = one_hot(np.argmax(victim_classifier.predict( adv_train_set[i].reshape( (1, adv_train_set[i].shape[ 0 ], adv_train_set[i].shape[ 1 ], adv_train_set[i].shape[ 2 ]) ) )))
    print("Inside Augment: " + str(oracle_labels.shape))
    
    # Create a session
    with tf.Session() as sess:
        xm = model_substitute.layers[0].input 
        yhat = wrapper.get_logits( xm )
        init = tf.global_variables_initializer()
        sess = tf.Session( )   
        sess.run(init) # Initializes the variables
        grads = sess.run(jacobian(yhat, xm, 10), feed_dict={xm: adv_train_set}) # compute the grads with the jacobian function
        jpd = jacobian_prediction_dimension(grads, np.argmax(oracle_labels, 1))

    perturbed_set = []
    jbda_lambda = 0.1
    tau = 1
    jbda_epoch_lambda = jbda_lambda * np.power(-1, np.floor(adv_train_epoch/tau))
    for idx, example in enumerate(adv_train_set):
        new_example = example + jbda_epoch_lambda * (np.sign(jpd[idx]))
        perturbed_set.append(new_example)
    adv_train_set = np.vstack((adv_train_set, np.array(perturbed_set)))

    print("After: ")
    print(adv_train_set.shape)

RUN: 0
Before: 
(7500, 32, 32, 3)
Oracle Predict: 
(7500, 10)
Epoch 1/10
58/58 [==============================] - 18s 313ms/step - loss: -4761129.9512 - accuracy: 0.9725
Epoch 2/10
58/58 [==============================] - 18s 305ms/step - loss: -3389399433.6552 - accuracy: 0.9922
Epoch 3/10
58/58 [==============================] - 18s 305ms/step - loss: -153465665483.0345 - accuracy: 0.9900
Epoch 4/10
58/58 [==============================] - 18s 306ms/step - loss: -2017703191940.4136 - accuracy: 0.9915
Epoch 5/10
58/58 [==============================] - 18s 306ms/step - loss: -13460708939846.6211 - accuracy: 0.9926
Epoch 6/10
58/58 [==============================] - 18s 306ms/step - loss: -58916139578615.1719 - accuracy: 0.9923
Epoch 7/10
58/58 [==============================] - 18s 306ms/step - loss: -194983216590989.2500 - accuracy: 0.9911
Epoch 8/10
58/58 [==============================] - 18s 305ms/step - loss: -530468002061488.5625 - accuracy: 0.9926
Epoch 9/10
58/58 [============

In [7]:
print(adv_train_set.shape)

(15000, 32, 32, 3)


In [8]:
# Collect subset
def exract_subset(data):
    x_pre = []  
    y_pre = []  
    #count = 0
    for index in range(0, len(data)):  
        x_pre.append(data[index])  
        y_predict = np.argmax(victim_classifier.predict( data[index].reshape( (1, data[index].shape[ 0 ], data[index].shape[ 1 ], data[index].shape[ 2 ]) ) ) )  # add the image label to the y_test set
        y_pre.append(y_predict)
        #if y_predict != np.argmax(labels[index]):
            #print(str(np.argmax(labels[index])) + " : " + str(y_predict))
            #count = count + 1
    #print(count)
    x = np.asarray(x_pre)  
    y = keras.utils.to_categorical(np.asarray(y_pre), 10)  
    return x, y

X_synthetic, Y_synthetic = exract_subset( adv_train_set )
print("X_synthetic shape: " + str(X_synthetic.shape) + "\n" + "X_synthetic size: " + str(X_synthetic.size) + "\n" +
      "Y_synthetic shape: " + str(Y_synthetic.shape) + "\n" + "Y_synthetic size: " + str(Y_synthetic.size) + "\n")

np.save('/content/gdrive/My Drive/X_synthetic', X_synthetic) 
np.save('/content/gdrive/My Drive/Y_synthetic', Y_synthetic) 

X_synthetic shape: (15000, 32, 32, 3)
X_synthetic size: 46080000
Y_synthetic shape: (15000, 10)
Y_synthetic size: 150000



In [9]:
X_train = np.load('/content/gdrive/My Drive/X_synthetic.npy') 
Y_train = np.load('/content/gdrive/My Drive/Y_synthetic.npy') 
print("X shape: " + str(X.shape) + "\n" + "X size: " + str(X.size) + "\n" + 
      "Y shape: " + str(Y.shape) + "\n" + "Y size: " + str(Y.size) + "\n")
print()

substitute_classifier.fit(X_train, Y_train, nb_epochs=10, batch_size=128)

X shape: (10000, 32, 32, 3)
X size: 30720000
Y shape: (10000, 10)
Y size: 100000


Epoch 1/10
117/117 [==============================] - 40s 345ms/step - loss: 164265293071.3163 - accuracy: 0.7601
Epoch 2/10
117/117 [==============================] - 36s 304ms/step - loss: 99132766855.6581 - accuracy: 0.7075
Epoch 3/10
117/117 [==============================] - 36s 305ms/step - loss: 62989470168.6154 - accuracy: 0.7147
Epoch 4/10
117/117 [==============================] - 36s 305ms/step - loss: 66073176186.5299 - accuracy: 0.7055
Epoch 5/10
117/117 [==============================] - 36s 306ms/step - loss: 53688011784.7521 - accuracy: 0.7092
Epoch 6/10
117/117 [==============================] - 36s 305ms/step - loss: 67378633281.6410 - accuracy: 0.7136
Epoch 7/10
117/117 [==============================] - 36s 306ms/step - loss: 27039552363.2137 - accuracy: 0.7087
Epoch 8/10
117/117 [==============================] - 37s 314ms/step - loss: 18308743010.4615 - accuracy: 0.6948
Epoch 9/10
1

In [10]:
#Evaluate the ART classifier on benign test examples
predictions = substitute_classifier.predict(x_test_substitute)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_substitute, axis=1)) / len(y_test_substitute)
print("Accuracy on benign test examples for substitute classifier: {}%".format(accuracy * 100))

Accuracy on benign test examples for substitute classifier: 10.639999999999999%


In [0]:
substitute_classifier.save("/content/gdrive/My Drive/Attack_Transfer_Substitute_Classifier_V3.h5")

In [13]:
!pip install adversarial-robustness-toolbox

In [0]:
import art
from art.classifiers import KerasClassifier
tf.compat.v1.disable_eager_execution()

In [0]:

from art.attacks import FastGradientMethod

fgsm = FastGradientMethod(classifier=substitute_classifier)
x_adversarial_fgsm = fgsm.generate(x_train_substitute,y_train_substitute)


In [29]:
predictions = substitute_classifier.predict(x_test_substitute)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_substitute, axis=1)) / len(y_test_substitute)
print("Accuracy on original examples for substitute classifier: {}%".format(accuracy))

Accuracy on original examples for substitute classifier: 0.1064%


In [27]:
predictions = substitute_classifier.predict(x_adversarial_fgsm)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_train_substitute, axis=1)) / len(y_train_substitute)
print("Accuracy on adversarial exales for substitute classifier:{}".format(accuracy))

Accuracy on adversarial exales for substitute classifier:0.09906666666666666


In [0]:
victim_classifier = KerasClassifier(model=model_substitute, clip_values=(0., 1.))

In [43]:
predictions = victim_classifier.predict(x_adversarial_fgsm) # giving the classifier the x_test of the CIFAR-10 dataset.
accuracy_benign = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_train_substitute, axis=1)) / len(y_train_substitute) # calculates the accuracy of the predictions
print("Accuracy on benign test examples for victim classifier: {}%\n".format(accuracy_benign * 100))


Accuracy on benign test examples for victim classifier: 9.906666666666666%



In [0]:
from art.attacks import SaliencyMapMethod

sm = SaliencyMapMethod(classifier=substitute_classifier)
x_adversarial_sm = fgsm.generate(x_train_substitute,y_train_substitute)

In [45]:
predictions = substitute_classifier.predict(x_adversarial_sm)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_train_substitute, axis=1)) / len(y_train_substitute)
print("Accuracy on original examples for substitute classifier: {}%".format(accuracy*100))

Accuracy on original examples for substitute classifier: 9.906666666666666%
